# Demystifying Input Image Pixel Normalization

# Contents
 1. [Outline](#outline)
 2. [MNIST Classifcation Task](#mnists_task)
 3. [CIFAR-10 Classification Task](#cifar10_task)

<a id='outline'></a>
# Outline

When dealing with images as well as other inputs to a deep neural network, it is important to normalize the pixel values. There are two main conventions: normalization to the range -1.0 to 1.0 and normalization to the range 0.0 to 1.0. 

The goal of this notebook is to explore this and find a way to train a netwrok without pixel normalization. First, the pixel normalization will be explored emperically in the context of the  MNIST and CIFAR-10 classification tasks.

In [ ]:
import sys
import os

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

sys.path.append(
    os.path.dirname(os.getcwd())
)
from utilities.tile_image_plot_utilities import\
    custom_tile_image_plot,\
    custom_tile_plot_with_inference_hists
from utilities.generator_utilities import ScrambledImageDataGenerator
from utilities.keras_callback_utilities import CustomHistory

<a id='mnist_task'></a>
<br><br><br>

----
# MNIST Classification Task

## Load Dataset

In [ ]:
mnist_train, mnist_test = tf.keras.datasets.mnist.load_data()

print("Train data:   ", mnist_train[0].shape)
print("Train labels: ", mnist_train[1].shape)
print("Test data:    ", mnist_test[0].shape)
print("Test labels:  ", mnist_test[1].shape)
print("--"*16)
print(f"Train data range: ({np.amin(mnist_train[0])}, {np.amax(mnist_train[0])})")
print(f"Test data range:  ({np.amin(mnist_test[0])}, {np.amax(mnist_test[0])})")

## Construct a Model

The model architecture is LeNet.

In [ ]:
# Define model:
tf.keras.backend.clear_session()
activation = "relu" # "sigmoid" 
mnist_model = tf.keras.Sequential(name="MNIST")
mnist_model.add(
    tf.keras.Input(shape=(28, 28, 1), batch_size=None, name="input")
)
mnist_model.add(
    tf.keras.layers.Conv2D(
        filters=6, 
        kernel_size=(5, 5), 
        strides=(1, 1), 
        activation=activation, 
        padding="same", 
        name="conv_1")
)
mnist_model.add(
    tf.keras.layers.AvgPool2D(pool_size=2, strides=2)
)
mnist_model.add(
    tf.keras.layers.Conv2D(
        filters=16, 
        kernel_size=(5, 5), 
        strides=(1, 1), 
        activation=activation, 
        padding="valid", 
        name="conv_2")
)
mnist_model.add(
    tf.keras.layers.AvgPool2D(pool_size=2, strides=2)
)
mnist_model.add(
    tf.keras.layers.Flatten()
)
mnist_model.add(
    tf.keras.layers.Dense(
        120, 
        activation=activation,
        name="dense_1")
)
mnist_model.add(
    tf.keras.layers.Dense(
        84, 
        activation=activation,
        name="dense_2")
)
mnist_model.add(
    tf.keras.layers.Dense(
        10, 
        activation="softmax",
        name="dense_3")
)

# Compile model:
mnist_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy")]
)

# Print model summary:
mnist_model.summary()

## Train Model

In [ ]:
normalize = not True
batch_size = 64
num_epochs = 20

data_generator = ScrambledImageDataGenerator(
    features=mnist_train[0][:, : ,: , np.newaxis],
    labels=mnist_train[1],
    batch_size=batch_size,
    scrambler_array=None,
    normalize=normalize)

custom_history = CustomHistory()

fit_history = mnist_model.fit(
    data_generator,
    epochs=num_epochs,
    verbose=1,
    callbacks=[
        custom_history,
        tf.keras.callbacks.History()],
    initial_epoch=0)

## Training History

In [ ]:
loss = fit_history.history["loss"]
accuracy = fit_history.history["accuracy"]
epochs = np.arange(1, len(loss)+1)

fig, ax1 = plt.subplots(figsize=(10., 6.))
ax2 = ax1.twinx()
#
color = "blue"
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Cross-entropy", fontsize=14., color=color)
ax1.plot(epochs, loss, ls="--", marker="d", color=color)
ax1.tick_params(axis="y", labelcolor=color)
ax1.set_xticks(epochs)
ax1.xaxis.grid()
ax1.set_ylim(bottom=0.0)
ax1.set_yscale("linear")
#
color = "gray"
ax2.set_ylabel("Accuracy", fontsize=14., color=color)
ax2.plot(epochs, accuracy, ls="--", marker="d", color=color)
ax2.tick_params(axis="y", labelcolor=color)
ax2.set_ylim(0.0, 1.0)
#
plt.show()

In [ ]:
loss_history = custom_history.get_loss_history()
epochs_list = list(loss_history.keys())
losses_array = np.array(list(loss_history.values())).flatten()
num_epochs = len(epochs_list)
num_points = int(np.prod(losses_array.shape))
num_batches = num_points // num_epochs
print(f"\tNumber of Epochs: {num_epochs}")
print(f"\tEpoch Size:       {num_batches}")
#
fig = plt.figure(figsize=(16., 8.))
x = np.linspace(
    start=epochs_list[0], 
    stop=epochs_list[-1]+1, 
    num=num_points, 
    endpoint=False)
plt.plot(x, losses_array, ls="-", lw=1.0, color="royalblue")
plt.xticks(epochs_list, labels=epochs_list)
plt.title("Full Training History", fontsize=14., fontweight="bold")
plt.ylabel("Loss (CE)", fontsize=12., fontweight="bold", color="gray")
plt.xlabel("Epochs", fontsize=12., fontweight="bold", color="gray")
plt.grid()
plt.gca().set_axisbelow(True)
#plt.yscale("symlog")
plt.show()

## Evaluation of the Test Set

<a id='cifar10_task'></a>
<br><br><br>

----
# CIFAR-10 Classifcation Task

## Load Dataset

In [ ]:
cifar10_train, cifar10_test = tf.keras.datasets.cifar10.load_data()
print("Train data:   ", cifar10_train[0].shape)
print("Train labels: ", cifar10_train[1].shape)
print("Test data:    ", cifar10_test[0].shape)
print("Test labels:  ", cifar10_test[1].shape)
print("--"*16)
print(f"Train data range: ({np.amin(cifar10_train[0])}, {np.amax(cifar10_train[0])})")
print(f"Test data range:  ({np.amin(cifar10_test[0])}, {np.amax(cifar10_test[0])})")

## Construct a Model

A CNN classifier.

In [ ]:
# Define model:
tf.keras.backend.clear_session()
activation = "relu" # "sigmoid"
cifar10_model = tf.keras.Sequential(name="CIFAR-10")
cifar10_model.add(
    tf.keras.Input(shape=(32, 32, 3), batch_size=None, name="input")
)
cifar10_model.add(
    tf.keras.layers.Conv2D(
        filters=6, 
        kernel_size=(5, 5), 
        strides=(1, 1), 
        activation=activation, 
        padding="same", 
        name="conv_1")
)
cifar10_model.add(
    tf.keras.layers.AvgPool2D(pool_size=2, strides=2)
)
cifar10_model.add(
    tf.keras.layers.Conv2D(
        filters=16, 
        kernel_size=(5, 5), 
        strides=(1, 1), 
        activation=activation, 
        padding="valid", 
        name="conv_2")
)
cifar10_model.add(
    tf.keras.layers.AvgPool2D(pool_size=2, strides=2)
)
cifar10_model.add(
    tf.keras.layers.Flatten()
)
cifar10_model.add(
    tf.keras.layers.Dense(
        120, 
        activation=activation,
        name="dense_1")
)
cifar10_model.add(
    tf.keras.layers.Dense(
        84, 
        activation=activation,
        name="dense_2")
)
cifar10_model.add(
    tf.keras.layers.Dense(
        10, 
        activation="softmax",
        name="dense_3")
)

# Compile model:
cifar10_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy")]
)

# Print model summary:
cifar10_model.summary()

## Train Model

In [ ]:
normalize = True
batch_size = 64
num_epochs = 50

data_generator = ScrambledImageDataGenerator(
    features=cifar10_train[0],
    labels=cifar10_train[1],
    batch_size=batch_size,
    scrambler_array=None,
    normalize=normalize)

custom_history = CustomHistory()

fit_history = cifar10_model.fit(
    data_generator,
    epochs=num_epochs,
    verbose=1,
    callbacks=[
        custom_history,
        tf.keras.callbacks.History()],
    initial_epoch=0)

## Training History

In [ ]:
loss = fit_history.history["loss"]
accuracy = fit_history.history["accuracy"]
epochs = np.arange(1, len(loss)+1)

fig, ax1 = plt.subplots(figsize=(10., 6.))
ax2 = ax1.twinx()
#
color = "blue"
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Cross-entropy", fontsize=14., color=color)
ax1.plot(epochs, loss, ls="--", marker="d", color=color)
ax1.tick_params(axis="y", labelcolor=color)
ax1.set_xticks(epochs)
ax1.xaxis.grid()
ax1.set_ylim(bottom=0.0)
ax1.set_yscale("linear")
#
color = "gray"
ax2.set_ylabel("Accuracy", fontsize=14., color=color)
ax2.plot(epochs, accuracy, ls="--", marker="d", color=color)
ax2.tick_params(axis="y", labelcolor=color)
ax2.set_ylim(0.0, 1.0)
#
plt.show()

In [ ]:
loss_history = custom_history.get_loss_history()
epochs_list = list(loss_history.keys())
losses_array = np.array(list(loss_history.values())).flatten()
num_epochs = len(epochs_list)
num_points = int(np.prod(losses_array.shape))
num_batches = num_points // num_epochs
print(f"\tNumber of Epochs: {num_epochs}")
print(f"\tEpoch Size:       {num_batches}")
#
fig = plt.figure(figsize=(16., 8.))
x = np.linspace(
    start=epochs_list[0], 
    stop=epochs_list[-1]+1, 
    num=num_points, 
    endpoint=False)
plt.plot(x, losses_array, ls="-", lw=1.0, color="royalblue")
plt.xticks(epochs_list, labels=epochs_list)
plt.title("Full Training History", fontsize=14., fontweight="bold")
plt.ylabel("Loss (CE)", fontsize=12., fontweight="bold", color="gray")
plt.xlabel("Epochs", fontsize=12., fontweight="bold", color="gray")
plt.grid()
plt.gca().set_axisbelow(True)
#plt.yscale("symlog")
plt.show()

## Evaluation of the Test Set